In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os, sys, itertools


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.params import ENRICH_ALPHA, MULTI_HYP_CORR_METHOD
pd.options.display.max_columns = 100

In [2]:
# exp_muts_df = pd.read_pickle("./data/9_df.pkl")  # Currently skipping imodulons for the sake of the AVA MS
exp_muts_df = pd.read_pickle("./data/8_df.pkl")
display(exp_muts_df.shape, exp_muts_df.head())

(114, 45)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
2527,28,None,[crl],DEL,257908,NC_000913,Δ776 bp,1,TOL_putrescine,50,1,1.0,1,True,"(257908, 258683)","{ECK120001080, ECK125229290, ECK125229291}","[{'name': 'insA9', 'RegulonDB ID': 'ECK1252292...",False,False,{},{},{},{},{},True,"[{'name': 'insB9', 'RegulonDB ID': 'ECK1252292...","{'ECK125229291': ['ECK125229291'], 'ECK1252292...",[],{},[],{},[],{},[],{},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2528,3,coding (163/210 nt),cspC,INS,1907273,NC_000913,(CGTCCTG)1→2,1,TOL_putrescine,50,1,1.0,1,True,"(1907273, 1907273)",{ECK120002092},"[{'name': 'cspC', 'RegulonDB ID': 'ECK12000209...",False,False,{},{},{},{},{},True,"[{'name': 'cspC', 'RegulonDB ID': 'ECK12000209...",{'ECK120002092': ['ECK120002092']},"[{'name': 'yobF-cspC', 'RegulonDB ID': 'ECK120...",{'ECK120028784': ['ECK120002092']},[],{},[],{},[{'name': 'Transcription'}],{'Transcription': ['ECK120028784']},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2529,29,None,"insB1, insA",DEL,1978503,NC_000913,Δ776 bp,1,TOL_putrescine,50,1,1.0,1,True,"(1978503, 1979278)","{ECK120003616, ECK120023915, ECK120003617}","[{'name': 'insB-5', 'RegulonDB ID': 'ECK120003...",False,False,"{ECK120017150, ECK120013472}","{ECK125095446, ECK120034252}",{},{},{},True,"[{'name': 'flhDC TFBS', 'RegulonDB ID': 'ECK12...","{'ECK120003616': ['ECK120013472', 'ECK12000361...","[{'name': 'flhDC', 'RegulonDB ID': 'ECK1200149...","{'ECK120014934': ['ECK120003616', 'ECK12000361...","[{'name': 'YjjQ', 'significantly associated co...","{'YjjQ': ['flhDC', 'flhDC'], 'Fur': ['flhDC', ...",[],{},"[{'name': 'Mobilome: prophages, transposons'},...","{'Mobilome: prophages, transposons': ['ECK1250...",37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2530,32,intergenic (‑1/+1),gatC/gatC,DEL,2173363,NC_000913,Δ2 bp,1,TOL_putrescine,50,1,1.0,1,False,"(2173363, 2173364)",{ECK120002276},"[{'name': 'gatC', 'RegulonDB ID': 'ECK12000227...",False,False,{},{},{},{},{},False,"[{'name': 'gatC', 'RegulonDB ID': 'ECK12000227...",{'ECK120002276': ['ECK120002276']},"[{'name': 'gatYZABCD', 'RegulonDB ID': 'ECK120...",{'ECK120015849': ['ECK120002276']},"[{'name': 'GatR', 'significantly associated co...","{'GatR': ['gatYZABCD'], 'ArcA': ['gatYZABCD'],...",[],{},[],{},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2531,4,N34K (AAT→AAG),mreB,SNP,3400986,NC_000913,A→C,1,TOL_putrescine,50,1,1.0,1,True,"(3400986, 3400986)",{ECK120000601},"[{'name': 'mreB', 'RegulonDB ID': 'ECK12000060...",False,False,{},{},{},{},{},True,"[{'name': 'mreB', 'RegulonDB ID': 'ECK12000060...",{'ECK120000601': ['ECK120000601']},"[{'name': 'mreBCD', 'RegulonDB ID': 'ECK120030...",{'ECK120030764': ['ECK120000601']},[],{},[],{},"[{'name': 'Cell cycle control, cell division, ...","{'Cell cycle control, cell division, chromosom...",37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)


# significance

In [3]:
# a bit behind annotations currently used in AVA MS.
# Still uses same approach to counted, just not all data-types have been consolidated into the COGs feat col


comp_mut_df = pd.DataFrame(columns=["observed mutation count"])
for i, r in exp_muts_df.iterrows():
    for f, links in r["COG links"].items():
        if "unknown" not in f:
            if f not in comp_mut_df.index:
                df = pd.DataFrame({"observed mutation count": len(links)}, index=[f])
                comp_mut_df = comp_mut_df.append(df)
            else:
                comp_mut_df.loc[f, "observed mutation count"] += len(links)
            
comp_mut_df

,observed mutation count
Transcription,22
"Mobilome: prophages, transposons",28
"Cell cycle control, cell division, chromosome partitioning",7
Carbohydrate transport and metabolism,6
Amino acid transport and metabolism,15
Cell motility,3
Inorganic ion transport and metabolism,10
Nucleotide transport and metabolism,3
Signal transduction mechanisms,4
"Translation, ribosomal structure and biogenesis",8


In [4]:
np.sum(comp_mut_df["observed mutation count"])

118

In [5]:
for _, r in exp_muts_df.iterrows():
    for feat_d in r["COGs"]:
        if "unknown" not in feat_d["name"]:
            feat_d["mutation set count"] = comp_mut_df.loc[feat_d["name"]]["observed mutation count"]

In [6]:
COG_len_df = pd.read_pickle("./data/COG_len_df.pkl")
COG_len_df.head()

,COG length
COG description,
Amino acid transport and metabolism,660007
Inorganic ion transport and metabolism,305849
Carbohydrate transport and metabolism,679809
Coenzyme transport and metabolism,212728
Energy production and conversion,589566


In [7]:
import numpy as np


comp_mut_df["length"] = comp_mut_df.apply(lambda r: COG_len_df.loc[r.index, "COG length"])
total_seq_len = np.sum(comp_mut_df["length"])
comp_mut_df["proportion"] = comp_mut_df["length"].apply(lambda comp_len: comp_len/total_seq_len)
comp_mut_df

,observed mutation count,length,proportion
Transcription,22,424603,0.083610
"Mobilome: prophages, transposons",28,28915,0.005694
"Cell cycle control, cell division, chromosome partitioning",7,115057,0.022656
Carbohydrate transport and metabolism,6,679809,0.133864
Amino acid transport and metabolism,15,660007,0.129965
Cell motility,3,144533,0.028461
Inorganic ion transport and metabolism,10,305849,0.060226
Nucleotide transport and metabolism,3,160339,0.031573
Signal transduction mechanisms,4,339255,0.066804
"Translation, ribosomal structure and biogenesis",8,393020,0.077391


In [8]:
from collections import Counter
import numpy as np
comp_mut_df['equal or larger count'] = 0
component_l = comp_mut_df.index.tolist()
proportion_l = comp_mut_df["proportion"].tolist()
num_muts = np.sum(comp_mut_df["observed mutation count"])

# slides from UW speak about the relationship of permutations and precision.
# should read through them to get a sense of what is the least iterations that should execute.
num_itr = 10000
for i in range(0, num_itr):
    random_mut_comp_a = np.random.choice(
        a=component_l,
        size=int(num_muts),
        p=proportion_l,
        replace=True,
    )
    random_mut_comp_count_d = Counter(random_mut_comp_a)
    for component in random_mut_comp_count_d.keys():
        if random_mut_comp_count_d[component] >= comp_mut_df.loc[component, 'observed mutation count']:
            comp_mut_df.loc[component, 'equal or larger count'] += 1

comp_mut_df["p value"] = comp_mut_df["equal or larger count"].apply(
    lambda count: count/num_itr)

comp_mut_df["significant"] = comp_mut_df["p value"].apply(lambda p_val: True if p_val < ENRICH_ALPHA else False)

display(comp_mut_df.head())

,observed mutation count,length,proportion,equal or larger count,p value,significant
Transcription,22,424603,0.083610,1,0.0001,True
"Mobilome: prophages, transposons",28,28915,0.005694,0,0.0000,True
"Cell cycle control, cell division, chromosome partitioning",7,115057,0.022656,201,0.0201,True
Carbohydrate transport and metabolism,6,679809,0.133864,9987,0.9987,False
Amino acid transport and metabolism,15,660007,0.129965,5820,0.5820,False


In [9]:
from statsmodels.stats import multitest
pvals = comp_mut_df["p value"]
rejects, pvals_corrected, alphacSidak, alphacBonf = multitest.multipletests(
    pvals=pvals,
    alpha=ENRICH_ALPHA,
    method=MULTI_HYP_CORR_METHOD)

pd.options.display.max_rows = 500
comp_mut_df["corrected p value"] = pvals_corrected
comp_mut_df["corrected significance"] = rejects
comp_mut_df.head()

,observed mutation count,length,proportion,equal or larger count,p value,significant,corrected p value,corrected significance
Transcription,22,424603,0.083610,1,0.0001,True,0.0015,True
"Mobilome: prophages, transposons",28,28915,0.005694,0,0.0000,True,0.0000,True
"Cell cycle control, cell division, chromosome partitioning",7,115057,0.022656,201,0.0201,True,0.3015,False
Carbohydrate transport and metabolism,6,679809,0.133864,9987,0.9987,False,1.0000,False
Amino acid transport and metabolism,15,660007,0.129965,5820,0.5820,False,1.0000,False


In [10]:
# # not wanting to consider mutation events with only 1 features as being significant
comp_mut_df["final significance"] = comp_mut_df.apply(lambda r: r["corrected significance"] if r["observed mutation count"] > 1 else False, axis=1)
# comp_mut_df.head()

In [11]:
for i, r in exp_muts_df.iterrows():
    for d in r["COGs"]:
        if d["name"] in comp_mut_df.index:
            d["significantly enriched"] = comp_mut_df.loc[d["name"]]["final significance"]

In [12]:
assert(len(exp_muts_df.exp.unique())==1)
exp = exp_muts_df.exp.unique()[0]
comp_mut_df.to_csv("./data/supp/permutation_test_results_per_experiment/"+exp+"_mut_COG_signif.csv")

In [13]:
exp_muts_df.to_pickle("./data/10_df.pkl")

In [14]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _get_name_signif_from_name(name, json):
#     is_signif = False
#     for d in json:
#         if d["name"] == name:
#             is_signif = d["significant"]
#             break
#     return is_signif


# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in exp_muts_df.iterrows():
#     link_d = r["COG links"]
#     for targ_name, hits in link_d.items():
#         is_signif = _get_name_signif_from_name(targ_name, r["COGs"])
#         t = str(targ_name)
#         if is_signif:
#             t = '*' + t
#         if t not in targ_cnt_d.keys():
#             targ_cnt_d[t] = 0
#         targ_cnt_d[t] += len(hits)
        
# df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# df = df[-20:]
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,10),
#                   color="#4C72B0",
#                   width=1
#                  )
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.2, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Top 20 mutated COGs")
# ax.get_legend().remove()

In [15]:
# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in exp_muts_df.iterrows():
#     link_d = r["COG links"]
#     for targ_name, hits in link_d.items():
#         is_signif = _get_name_signif_from_name(targ_name, r["COGs"])
#         t = str(targ_name)
#         if is_signif:
#             t = '*' + t
#             if t not in targ_cnt_d.keys():
#                 targ_cnt_d[t] = 0
#             targ_cnt_d[t] += len(hits)
# if len(targ_cnt_d) > 0:
#     df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
#     df = df.sort_values("mutated feature count")
#     df = df[-20:]
#     ax = df.plot.barh(y='mutated feature count',
#     #                   figsize=(10,25),
#     #                   figsize=(5,10),
#                       color="#4C72B0",
#                       width=1
#                      )
#     for i, v in enumerate(df["mutated feature count"]):
#         ax.text(v, i - 0.2, str(v))
#     sns.despine(ax=ax, top=True, right=True)
#     ax.set_title("Signifcant mutated COGs")
#     ax.get_legend().remove()